In [1]:
# Data
import pandas as pd
import numpy as np
import datetime

# Notebook Settings
import os

In [2]:
# set wd
os.getcwd()
os.chdir('/home/ad-frazier/data_science/MSBA_320/final_project')

# set pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# Read in helper files

# State Full name to abbreviation
state_dict = pd.read_csv('https://github.com/adfrisealach/msba_320_fp/blob/main/helpers/states.csv?raw=true', header=None, skiprows=1 , index_col=0, squeeze=True).to_dict()

Files cleaned in order of perceived importance

# Financial Data

### GDP by State and Year

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/dirty/1_gdp_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [5]:
df.head(2)

,GeoFips,GeoName,LineCode,Description,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,United States,3,Current-dollar GDP (millions of current dollars),8577552.00,9062817.00,9631172.00,10250952.00,10581929.00,10929108.00,11456450.00,12217196.00,13039197.00,13815583.00,14474228.00,14769862.00,14478067.00,15048970.00,15599731.00,16253970.00,16843196.00,17550687.00,18206023.00,18695106.00,19479623.00,20527159.00,21372582.00,20893746.00
1,0,United States,4,Compensation (millions of dollars),4713220.00,5075701.00,5409937.00,5854634.00,6046346.00,6143370.00,6362298.00,6729306.00,7077722.00,7491260.00,7889371.00,8068682.00,7767191.00,7932970.00,8234017.00,8575362.00,8843637.00,9259654.00,9709535.00,9977096.00,10436745.00,10969807.00,11459449.00,11580088.00


In [6]:
# reshape
df = pd.melt(df, id_vars=['GeoFips','GeoName','LineCode','Description'])

In [7]:
# rename
df = df.rename(columns={'variable':'year'}) 

#force state to lowercase
df['GeoName'] = df['GeoName'].str.upper()

df['state'] = df['GeoName'].map(state_dict)

df = df.drop(columns='GeoName')

In [8]:
df['Description'].value_counts()

Current-dollar GDP (millions of current dollars)                  1440
Compensation (millions of dollars)                                1440
Gross operating surplus (millions of dollars)                     1440
  Taxes on production and imports (TOPI) (millions of dollars)    1440
  Subsidies (millions of dollars)                                 1440
Name: Description, dtype: int64

In [9]:
df.head(2)

,GeoFips,LineCode,Description,year,value,state
0,0,3,Current-dollar GDP (millions of current dollars),1997,8577552.00,NaN
1,0,4,Compensation (millions of dollars),1997,4713220.00,NaN


In [10]:
# # re-organize description format + convert to actual dollar format 
# df['gdp_current_dollar'] = df['value'].loc[df['Description'] == 'Current-dollar GDP (millions of current dollars)']*1000000

# df['gdp_compensation'] = df['value'].loc[df['Description'] == 'Compensation (millions of dollars)']*1000000

# df['gdp_gross_surplus'] = df['value'].loc[df['Description'] == 'Gross operating surplus (millions of dollars)']*1000000

# df['gdp_production_import_taxes'] = df['value'].loc[df['Description'] == 'Taxes on production and imports (TOPI) (millions of dollars)']*1000000

# df['gdp_subsidies'] = df['value'].loc[df['Description'] == 'Subsidies (millions of dollars)']*1000000

# drop uneeded columns
df = df.drop(columns=['LineCode'])

# keep original description + value to allow categorical plotting in python
df = df.rename(columns={'Description':'og_gdp_description', 'value':'og_gdp_value'})

df['og_gdp_value'] = df['og_gdp_value']*1000000

In [11]:
df.sample(5)

,GeoFips,og_gdp_description,year,og_gdp_value,state
6594,97000,Subsidies (millions of dollars),2018,-2410500000.00,NaN
5731,8000,Compensation (millions of dollars),2016,182139900000.00,CO
6199,42000,Subsidies (millions of dollars),2017,-1974700000.00,PA
222,48000,Gross operating surplus (millions of dollars),1997,244940900000.00,TX
5122,5000,Gross operating surplus (millions of dollars),2014,45138100000.00,AR


In [12]:
df.groupby('og_gdp_description')['og_gdp_value'].describe()

,count,mean,std,min,25%,50%,75%,max
og_gdp_description,,,,,,,,
Subsidies (millions of dollars),1440.00,-4172445138.89,22920330562.87,-761388000000.00,-2047075000.00,-935500000.00,-427150000.00,-37400000.00
Taxes on production and imports (TOPI) (millions of dollars),1440.00,52211993680.56,144409964137.35,1421900000.00,5792450000.00,14753750000.00,37139800000.00,1534651000000.00
Compensation (millions of dollars),1440.00,398464107083.33,1095393650308.04,6990900000.00,44519075000.00,120593100000.00,295295875000.00,11580088000000.00
Current-dollar GDP (millions of current dollars),1440.00,733853649305.56,2029844104237.38,14654500000.00,81097625000.00,218582500000.00,530894975000.00,21372582000000.00
Gross operating surplus (millions of dollars),1440.00,287349978194.44,803666663448.03,5199000000.00,31002875000.00,85088200000.00,200848075000.00,8540393000000.00


In [13]:
df['og_gdp_description'] = df['og_gdp_description'].str.replace("(millions of dollars)", "", regex=False)
df['og_gdp_description'] = df['og_gdp_description'].str.replace("(millions of current dollars)", "", regex=False)

In [14]:
df.to_csv('./clean/gdp_state_annual_clean.csv', index=False)

### PCE State Annual

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/dirty/pce_state_annual.csv', skipfooter=4, engine='python')
# verified correct head/tail skip against manual excel inspections

In [16]:
df.head(2)

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"""00000""",United States,,SAPCE1,1,...,Personal consumption expenditures,Millions of current dollars,5536790.00,5877248.00,6283758.00,6767179.00,7073801.00,7348941.00,7740749.00,8231960.00,8769066.00,9277236.00,9746594.00,10050083.00,9891218.00,10260256.00,10698857.00,11047363.00,11363528.00,11847725.00,12263476.00,12693266.00,13239111.00,13913531.00,14428676.00,14047565.00
1,"""00000""",United States,,SAPCE1,2,...,Goods,Millions of current dollars,2006506.00,2108439.00,2287062.00,2453172.00,2525593.00,2598805.00,2722597.00,2902021.00,3082923.00,3239655.00,3367031.00,3363221.00,3180022.00,3317825.00,3518121.00,3637739.00,3729973.00,3862956.00,3922993.00,3991849.00,4158554.00,4353716.00,4478918.00,4653822.00


In [17]:
# drop initially unnecessary columns
df = df.drop(columns=['Region','TableName','LineCode','IndustryClassification'])

# reshape column structure
df = pd.melt(df, id_vars=['GeoFIPS','GeoName','Description','Unit'])

#force state to lowercase
df['GeoName'] = df['GeoName'].str.upper()

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

# rename columns
df = df.rename(columns={'variable':'year', 'Description':'og_pce_description','value':'og_pce_value'})

# Convert to 1:1 dollars
df['og_pce_value'] = df['og_pce_value']*1000000

In [18]:
df.head(2)

,GeoFIPS,GeoName,og_pce_description,Unit,year,og_pce_value,state
0,"""00000""",UNITED STATES,Personal consumption expenditures,Millions of current dollars,1997,5536790000000.00,NaN
1,"""00000""",UNITED STATES,Goods,Millions of current dollars,1997,2006506000000.00,NaN


In [19]:
# final drop of unnecessary columns
df = df.drop(columns=['GeoName','Unit'])

In [20]:
df.to_csv('./clean/pce_state_annual_clean.csv', index=False)

### Per Capita Disposable Personal Income

In [21]:
df = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/dirty/per_capita_disposable_personal_income_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [22]:
df.sample(5)

,GeoFips,GeoName,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
3,4000,Arizona,20080,21189,21855,23048,23731,24599,25748,27367,28904,30918,31988,32197,30950,31131,31919,32982,33141,34485,35547,36433,37923,39629,41166,45025
12,15000,Hawaii *,23298,23576,24434,25738,26583,27993,29269,30888,32416,34225,35999,37522,37537,38109,39031,40054,39859,41878,43237,44540,46457,47818,49371,52872
43,47000,Tennessee,20856,22556,23182,24367,24772,25758,26867,28219,29027,30184,31101,32127,32261,33503,34985,36559,36379,37502,38982,39762,41315,43455,45085,47192
29,32000,Nevada,24183,25613,26557,27871,28484,28707,29837,31545,33742,35271,35704,34880,33198,34393,35332,36223,35468,37525,39774,40234,42351,44302,46083,48767
37,40000,Oklahoma,18590,19379,19713,21278,22552,23424,24727,26693,28823,31355,31956,35341,33064,34325,36150,38400,39896,42262,40529,38415,40209,42272,44067,45727


In [23]:
# reshape column structure
df = pd.melt(df, id_vars=['GeoFips','GeoName'])

In [24]:
# rename columns
df = df.rename(columns={'variable':'year', 'value':'per_capita_disposable_personal_income'})

In [25]:
#force state to lowercase
df['GeoName'] = df['GeoName'].str.upper()

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

In [26]:
df = df.drop(columns=['GeoName'])

In [27]:
df.to_csv('./clean/per_capita_disposable_personal_income_state_annual_clean.csv', index=False)

### Per Capita Personal Income

In [28]:
df = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/dirty/per_capita_personal_income_state_annual.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [29]:
df.sample(5)

,GeoFips,GeoName,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
44,48000,Texas,23919,25556,26530,28383,29644,29524,30224,31195,33276,35630,37085,39946,37269,39029,42011,44193,44745,47273,47345,46445,49055,51813,53266,55129
4,5000,Arkansas,20187,21208,21871,22781,23873,24306,25595,27059,28227,29617,31303,32306,31472,32367,34253,36533,36605,38650,39842,40720,41873,43446,44788,47235
15,18000,Indiana,24096,25790,26600,28153,28639,28918,29521,30791,31566,33123,34055,35408,34267,35699,37822,39477,39829,41274,42778,43780,45278,47292,48980,51926
3,4000,Arizona,22804,24173,24960,26388,27008,27353,28393,30221,32327,34703,35872,35748,33524,33848,35175,36272,36739,38211,39635,40618,42267,44071,45808,49648
42,46000,South Dakota,22582,24277,25306,26825,27645,27572,29972,32025,33788,35299,38748,41168,39518,41417,44726,45384,45525,47374,49040,49018,50145,52346,54725,59281


In [30]:
# reshape column structure
df = pd.melt(df, id_vars=['GeoFips','GeoName'])

# rename columns
df = df.rename(columns={'variable':'year', 'value':'per_capita_personal_income'})

#force state to lowercase
df['GeoName'] = df['GeoName'].str.upper()

# map state name to state abbreviation
df['state'] = df['GeoName'].map(state_dict)

df = df.drop(columns=['GeoName'])

In [31]:
df.sample(5)

,GeoFips,year,per_capita_personal_income,state
136,19000,1999,25740,IA
520,44000,2005,36984,RI
775,94000,2009,38687,NaN
641,45000,2007,32402,SC
787,9000,2010,61762,CT


In [32]:
df.to_csv('./clean/per_capita_personal_income_state_annual_clean.csv', index=False)

### Business Applications State Annual

In [33]:
df = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/dirty/business_apps_monthly_structured.csv', engine='python')
# verified correct head/tail skip against manual excel inspections

In [34]:
# State Full name to abbreviation
naic_sector_codes = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/helpers/naic_sectors_codes.csv', header=None, skiprows=1 , index_col=0, squeeze=True).to_dict()

naic_series_codes = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/helpers/naic_series_codes.csv', header=None, skiprows=1 , index_col=0, squeeze=True).to_dict()

In [35]:
df.tail(5)

,sa,naics_sector,series,geo,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
29711,U,TOTAL,BF_DUR8Q,VA,2004,NaN,NaN,NaN,NaN,NaN,NaN,1.31,1.38,1.74,1.45,1.41,1.87
29712,U,TOTAL,BF_DUR8Q,WA,2004,NaN,NaN,NaN,NaN,NaN,NaN,1.07,1.22,1.51,1.15,1.19,1.5
29713,U,TOTAL,BF_DUR8Q,WV,2004,NaN,NaN,NaN,NaN,NaN,NaN,1.03,1.26,1.49,0.94,1.02,1.39
29714,U,TOTAL,BF_DUR8Q,WI,2004,NaN,NaN,NaN,NaN,NaN,NaN,1.07,1.35,1.62,1.35,1.3,1.72
29715,U,TOTAL,BF_DUR8Q,WY,2004,NaN,NaN,NaN,NaN,NaN,NaN,0.81,0.91,1.47,1.07,0.89,1.48


In [36]:
# Filter out incomplete years
df = df[df['year']!=2022]

df = df[df['year']!=2004]

# filter out unneccesary series
df = df[~df['series'].isin(['BF_PBF4Q','BF_PBF8Q','BF_SBF4Q','BF_SBF8Q'])]

In [37]:
df.tail(5)

,sa,naics_sector,series,geo,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
28039,U,TOTAL,BF_DUR8Q,VA,2005,1.09,1.33,1.68,1.28,1.47,1.79,1.36,1.42,1.8,1.35,1.64,1.82
28040,U,TOTAL,BF_DUR8Q,WA,2005,0.98,1.2,1.47,1.15,1.31,1.51,1.04,1.24,1.48,1.11,1.29,1.69
28041,U,TOTAL,BF_DUR8Q,WV,2005,0.62,1.19,1.4,1.13,1.25,1.58,0.92,1.09,1.38,1.33,1.2,1.46
28042,U,TOTAL,BF_DUR8Q,WI,2005,1.04,1.18,1.63,0.97,1.28,1.6,1.16,1.15,1.53,1.11,1.32,1.78
28043,U,TOTAL,BF_DUR8Q,WY,2005,0.73,1.16,1.56,1.17,1.05,1.44,0.76,1.17,1.37,0.94,1.18,1.7


In [38]:
# convert selected columns by index(months) to numeric
df.iloc[:,5:17] = df.iloc[:,5:17].apply(pd.to_numeric, errors='coerce')

# create annual total
df['annual_total'] =  df.iloc[:,5:17].sum(axis=1, min_count=12)

# drop monthly columns after creating annual total
df = df.drop(df.columns[5:17], axis=1)

In [39]:
# map the stuffs
df['naics_sector'] = df['naics_sector'].map(naic_sector_codes)

df['series'] = df['series'].map(naic_series_codes)

In [40]:
df.tail(5)

,sa,naics_sector,series,geo,year,annual_total
28039,U,Total for All NAICS,Average Duration (in Quarters) from Business A...,VA,2005,18.03
28040,U,Total for All NAICS,Average Duration (in Quarters) from Business A...,WA,2005,15.47
28041,U,Total for All NAICS,Average Duration (in Quarters) from Business A...,WV,2005,14.55
28042,U,Total for All NAICS,Average Duration (in Quarters) from Business A...,WI,2005,15.75
28043,U,Total for All NAICS,Average Duration (in Quarters) from Business A...,WY,2005,14.23


In [41]:
df.to_csv('./clean/business_apps_monthly_structured_clean.csv', index=False)

### Poverty Rate

In [71]:
df = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/dirty/poverty_rate_state_annual.csv ', engine='python')
# verified correct head/tail skip against manual excel inspections

In [80]:
df.sample(2)

,state,percent,year
799,ND,0.11,2005
689,MT,0.13,2007
718,CA,0.12,2006
136,ND,0.10,2018
954,OK,0.14,2002


In [73]:
# drop unused columns
df = df.drop(columns='Unnamed: 3')

# convert to numeric percent
df['Percent'] = (df['Percent']/100)

In [74]:
# convert state to upper for mapping
df['STATE'] = df['STATE'].str.upper()

df['STATE'] = df['STATE'].map(state_dict)

# convert column names to lower case
df.columns = df.columns.str.lower()

In [81]:
df.sample(2)

,state,percent,year
520,GA,0.19,2010
587,MT,0.14,2009


In [82]:
df.to_csv('./clean/poverty_rate_state_annua_clean.csv', index=False)

# Education Data

### Total Spending per Student

In [42]:
df = pd.read_csv('https://raw.githubusercontent.com/adfrisealach/msba_320_fp/main/dirty/tot_spend_student_state_annual.csv', skiprows=6, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [43]:
# Create String
string = 'Total Current Expenditures for Public El-Sec (TE5) per Pupil (MEMBR) [State Finance]'

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

In [44]:
df.sample(1)

,State Name,2018-19,2017-18,2016-17,2015-16,2014-15,2013-14,2012-13,2011-12,2010-11,2009-10,2008-09,2007-08,2006-07,2005-06,2004-05,2003-04,2002-03,2001-02,2000-01,1999-00,1998-99,1997-98
29,NEW HAMPSHIRE,17457,16588,15958,15535,14969,14601,14050,13774,13548,13072,12583,11951,11036,10396,9771,9161,8579,7935,7286,6860,6433,6156


In [45]:
# reshape dataframe
df = pd.melt(df, id_vars=['State Name'])

#rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_expenditure'})

# trim to desired year digits
df['start_year'] = df['start_year'].str[:5]

#force state to lowercase
df['state'] = df['state'].str.upper()

# map state name to state abbreviation
df['state'] = df['state'].map(state_dict)

In [46]:
# Check output
df.head(10)

,state,start_year,per_pupil_expenditure
0,AL,2018,10107
1,AK,2018,18393
2,AZ,2018,8773
3,AR,2018,10412
4,CA,2018,13831
5,CO,2018,11072
6,CT,2018,21140
7,DE,2018,15929
8,DC,2018,22831
9,FL,2018,9986


In [47]:
df.to_csv('./clean/tot_spend_student_state_annual_clean.csv', index=False)

### Total Revenue Per Student

In [48]:
df = pd.read_csv('./raw/tot_rev_per_student_state_annual.csv', skiprows=0, skipfooter=7, engine='python')
# verified correct head/tail skip against manual excel inspections

In [49]:
df.head(5)

,State Name,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2018-19,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2017-18,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2016-17,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2015-16,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2014-15,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2013-14,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2012-13,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2011-12,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2010-11,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2009-10,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2008-09,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2007-08,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2006-07,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2005-06,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2004-05,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2003-04,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2002-03,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2001-02,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 2000-01,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1999-00,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1998-99,Total Revenues (TR) per Pupil (MEMBR) [State Finance] 1997-98
0,ALABAMA,11415,10792,10590,10200,9992,9913,9653,9534,9776,9667,9708,10356,9548,8555,8028,7349,6971,6956,6503,6523,5975,5535
1,ALASKA,19553,19038,18897,18851,22379,20447,20312,19034,18699,17759,17319,17471,14304,12849,12632,11576,10928,10717,10275,10118,9532,9222
2,ARIZONA,10396,9697,9214,9293,8995,8703,8616,8613,9111,9343,9002,9457,9023,8071,7814,7550,7791,7214,7071,6455,5988,5812
3,ARKANSAS,11733,11592,11388,10939,10762,10478,10391,10939,10939,10738,10072,9758,9362,9031,8712,7542,7243,7112,6250,6054,5772,5697
4,CALIFORNIA,16014,14535,13965,13606,11786,10985,10481,10590,10790,10239,11180,11228,10857,9909,9234,8980,8975,8363,8306,7465,6750,6572


In [50]:
# Create String
string = 'Total Revenues (TR) per Pupil (MEMBR) [State Finance] '

# Left Strip String
df.columns = df.columns.str.replace(string, '', regex=False)

# Reshape df
df = pd.melt(df, id_vars=['State Name'])

# Rename columns
df = df.rename(columns={'State Name':'state', 'variable':'start_year','value':'per_pupil_revenue'})

# force state to uppercase
df['state'] = df['state'].str.upper()

# remap state name to state abbreviations
df['state'] = df['state'].map(state_dict)

# Use Start year instead of year-span
df['start_year'] = df['start_year'].str[:4]

In [51]:
# Sample output
df.sample(10)

,state,start_year,per_pupil_revenue
290,OH,2013,13627
1101,NJ,1997,10550
616,CA,2006,10857
703,SC,2005,9559
332,MT,2012,11601
68,KY,2017,12422
606,VT,2007,16000
191,PA,2015,17405
1003,ND,1999,6651
1082,HI,1997,6755


In [52]:
df.to_csv('./clean/tot_rev_per_student_state_annual_clean.csv', index=False)